In [1]:
import pandas as pd
import json
from konlpy.tag import Okt
import numpy as np
import ast

# --- 1. SentiWord_info.json 파일 "한 번만" 불러와 딕셔너리로 만들기 ---
senti_dic_path = './KNU/KnuSentiLex/data/SentiWord_info.json'  # 사용자가 업로드한 파일 경로
try:
    with open(senti_dic_path, 'r', encoding='utf-8') as f:
        senti_data = json.load(f)
    # 파일을 성공적으로 읽었으므로, {단어: 점수} 형태의 딕셔너리로 변환합니다.
    senti_dict = {item['word']: int(item['polarity']) for item in senti_data}
    print(f"SentiWord_info.json 감성사전을 성공적으로 불러왔습니다. (총 {len(senti_dict)}개 단어)")
except FileNotFoundError:
    print(f"오류: '{senti_dic_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    senti_dict = {}
except json.JSONDecodeError:
    print(f"오류: '{senti_dic_path}' 파일이 올바른 JSON 형식이 아닙니다.")
    senti_dict = {}


# --- 2. 분석할 데이터 불러오기 ---
# 이전 단계에서 문장 추출이 완료된 CSV 파일 경로
cheongju_analyzed_path = "./crawled_data/merged_cheongju_analyzed_fnai.csv"
other_analyzed_path = "./crawled_data/merged_other_analyzed_fnai.csv"

try:
    cheongju_df = pd.read_csv(cheongju_analyzed_path, encoding='utf-8-sig')
    other_df = pd.read_csv(other_analyzed_path, encoding='utf-8-sig')
    print("분석할 CSV 파일들을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: 분석할 CSV 파일을 찾을 수 없습니다. 이전 단계가 정상적으로 완료되었는지 확인해주세요.")
    cheongju_df = pd.DataFrame()
    other_df = pd.DataFrame()

okt = Okt()
print("="*50)


# --- 3. 카테고리별 점수 계산 함수 정의 ---
def calculate_category_scores_efficiently(row):
    categories = ['음식', '분위기', '가격', '서비스']
    final_scores = {}

    for category in categories:
        sentence_column = f'{category}_문장'
        try:
            sentences = ast.literal_eval(row[sentence_column]) if isinstance(row[sentence_column], str) and row[sentence_column].startswith('[') else []
        except:
            sentences = []

        if not sentences:
            final_scores[f'{category}_점수'] = 0
            continue

        category_total_score = 0
        sentence_count_with_sentiment = 0
        
        for sentence in sentences:
            try:
                morphs = okt.pos(sentence, stem=True)
                words = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']]
            except:
                words = []
            
            sentence_score = 0
            sentiment_word_count = 0
            for word in words:
                # 미리 만들어 둔 senti_dict에서 직접 점수를 조회합니다 (파일을 다시 읽지 않아 매우 빠름)
                if word in senti_dict:
                    sentence_score += senti_dict[word]
                    sentiment_word_count += 1
            
            if sentiment_word_count > 0:
                category_total_score += sentence_score
                sentence_count_with_sentiment += 1

        if sentence_count_with_sentiment > 0:
            final_scores[f'{category}_점수'] = category_total_score / sentence_count_with_sentiment
        else:
            final_scores[f'{category}_점수'] = 0 # 감성 단어가 없는 경우 0점(중립)으로 처리

    return pd.Series(final_scores)

# --- 4. 데이터프레임에 함수 적용 ---
print("카테고리별 감성 점수 계산 시작...")

if not cheongju_df.empty and senti_dict:
    cheongju_scores = cheongju_df.apply(calculate_category_scores_efficiently, axis=1)
    cheongju_final = pd.concat([cheongju_df, cheongju_scores], axis=1)
    print("'cheongju' 데이터프레임 점수 계산 완료.")

if not other_df.empty and senti_dict:
    other_scores = other_df.apply(calculate_category_scores_efficiently, axis=1)
    other_final = pd.concat([other_df, other_scores], axis=1)
    print("'other' 데이터프레임 점수 계산 완료.")
print("="*50)

# --- 5. 최종 결과 확인 및 저장 ---
output_cheongju_path_final = "./crawled_data/merged_cheongju_fnai_final.csv"
output_other_path_final = "./crawled_data/merged_other_fnai_final.csv"

print("최종 분석 결과를 새로운 CSV 파일로 저장합니다...")

if 'cheongju_final' in locals():
    cheongju_final.to_csv(output_cheongju_path_final, encoding='utf-8-sig', index=False)
    print(f"청주 데이터 최종 결과가 '{output_cheongju_path_final}' 파일에 저장되었습니다.")

if 'other_final' in locals():
    other_final.to_csv(output_other_path_final, encoding='utf-8-sig', index=False)
    print(f"다른 지역 데이터 최종 결과가 '{output_other_path_final}' 파일에 저장되었습니다.")
print("="*50)

# --- 최종 결과 데이터프레임 일부 출력 ---
print("\n--- 청주 데이터프레임 최종 분석 결과 (상위 5개) ---")
if 'cheongju_final' in locals():
    display_cols = ['음식_문장', '음식_점수', '분위기_문장', '분위기_점수', '가격_문장', '가격_점수', '서비스_문장', '서비스_점수']
    print(cheongju_final[display_cols].head())

SentiWord_info.json 감성사전을 성공적으로 불러왔습니다. (총 14852개 단어)
분석할 CSV 파일들을 성공적으로 불러왔습니다.
카테고리별 감성 점수 계산 시작...
'cheongju' 데이터프레임 점수 계산 완료.
'other' 데이터프레임 점수 계산 완료.
최종 분석 결과를 새로운 CSV 파일로 저장합니다...
청주 데이터 최종 결과가 './crawled_data/merged_cheongju_fnai_final.csv' 파일에 저장되었습니다.
다른 지역 데이터 최종 결과가 './crawled_data/merged_other_fnai_final.csv' 파일에 저장되었습니다.

--- 청주 데이터프레임 최종 분석 결과 (상위 5개) ---
                                               음식_문장     음식_점수  \
0  ['꾸덕한 빠네 소스가 적당합니다.', '동백카츠는 와사비와 홀그레인 소스로 눈을 ...  2.666667   
1  ['메뉴판은 종이 메뉴판을 찍어왔어요미소야는 역시 다양한 일식 메뉴가있다는 점이 참...  3.833333   
2  ['기다리면서메뉴 정독하면서 골랐다.', '무튼 그래서2인 3메뉴 가자조합을 어떻게...  0.750000   
3  ['청주까지 찾아와준 친구와이른 저녁식사를 하기 위해오창에 소재한 회전 초밥집 스시...  1.857143   
4  ['지웰시티 데이트 하기 좋은 분위기 맛집 스테이크 덮밥부터 파스타 곱창라멘까지 완...  2.800000   

                                              분위기_문장  분위기_점수  \
0  ['한줄 평 연인 가족들과 데이트 하기 참 좋은 식당 지극히 개인적인 의견일 뿐입니다']     2.0   
1                                                 []     0.0   
2                         ['계속 고민을 열심히 

In [2]:
# 최대 점수 출력
print("----- 청주 최대 점수 출력 -----")
if 'cheongju_final' in locals():
    max_scores = cheongju_final[['음식_점수', '분위기_점수', '가격_점수', '서비스_점수']].max()
    print("청주 데이터 최대 점수:")
    print(max_scores)

print("\n--- 다른 지역 최대 점수 출력 ---")
if 'other_final' in locals():
    max_other_scores = other_final[['음식_점수', '분위기_점수', '가격_점수', '서비스_점수']].max()
    print("다른 지역 데이터 최대 점수:")
    print(max_other_scores)

print("="*50)

# 최소 점수 출력
print("----- 청주 최소 점수 출력 -----")
if 'cheongju_final' in locals():
    min_scores = cheongju_final[['음식_점수', '분위기_점수', '가격_점수', '서비스_점수']].min()
    print("청주 데이터 최소 점수:")
    print(min_scores)
print("\n--- 다른 지역 최소 점수 출력 ---")
if 'other_final' in locals():
    min_other_scores = other_final[['음식_점수', '분위기_점수', '가격_점수', '서비스_점수']].min()
    print("다른 지역 데이터 최소 점수:")
    print(min_other_scores)

----- 청주 최대 점수 출력 -----
청주 데이터 최대 점수:
음식_점수     19.0
분위기_점수    25.0
가격_점수     17.0
서비스_점수    24.0
dtype: float64

--- 다른 지역 최대 점수 출력 ---
다른 지역 데이터 최대 점수:
음식_점수     22.0
분위기_점수    48.0
가격_점수     48.0
서비스_점수    23.5
dtype: float64
----- 청주 최소 점수 출력 -----
청주 데이터 최소 점수:
음식_점수    -2.0
분위기_점수   -3.0
가격_점수    -4.0
서비스_점수   -4.0
dtype: float64

--- 다른 지역 최소 점수 출력 ---
다른 지역 데이터 최소 점수:
음식_점수    -6.0
분위기_점수   -7.0
가격_점수    -7.0
서비스_점수   -5.0
dtype: float64


In [3]:
# 최소 점수 문장 출력
print("----- 청주 최소 점수 문장 출력 -----")
if 'cheongju_final' in locals():
    min_sentences = cheongju_final.loc[cheongju_final[['음식_점수', '분위기_점수', '가격_점수', '서비스_점수']].min(axis=1).idxmin()]
    print("청주 데이터 최소 점수 문장:")
    print(min_sentences[['음식_문장', '분위기_문장', '가격_문장', '서비스_문장']])

print("\n--- 다른 지역 최소 점수 문장 출력 ---")
if 'other_final' in locals():
    min_other_sentences = other_final.loc[other_final[['음식_점수', '분위기_점수', '가격_점수', '서비스_점수']].min(axis=1).idxmin()]
    print("다른 지역 데이터 최소 점수 문장:")
    print(min_other_sentences[['음식_문장', '분위기_문장', '가격_문장', '서비스_문장']])

----- 청주 최소 점수 문장 출력 -----
청주 데이터 최소 점수 문장:
음식_문장     ['무슨 가게가 있는지 라인업이랑구조가 한눈에 다 보여용 입구 양 옆으로는푸드트럭들...
분위기_문장    ['요게 이제 6월말에 하던걸다시 한다는 느낌 요게 이제 육거리시장 야시장 부스안으...
가격_문장     ['오늘은 지난 여름에 이은청주 육거리시장 만원에오픈런 하러 왔습니다', '9월19...
서비스_문장    ['아니 근디 먹고있는디갑자기 백로식당 부스에서직원분이 이거 주고 가시는거임그래서 ...
Name: 433, dtype: object

--- 다른 지역 최소 점수 문장 출력 ---
다른 지역 데이터 최소 점수 문장:
음식_문장     ['몸도 마음도 가볍게 떠나는 옥천 하루 코스 코스 옥천 구읍 골목투어 구읍 벚꽃길...
분위기_문장    ['몸도 마음도 가볍게 떠나는 옥천 하루 코스 코스 옥천 구읍 골목투어 구읍 벚꽃길...
가격_문장     ['몸도 마음도 가볍게 떠나는 옥천 하루 코스 코스 옥천 구읍 골목투어 구읍 벚꽃길...
서비스_문장                                                   []
Name: 624, dtype: object


In [4]:
cheongju_final.to_excel("./crawled_data/merged_cheongju_final.xlsx", index=False)